# 🤖 Скачивание локальной LLM модели для RAG системы

Этот ноутбук поможет вам скачать и настроить локальную LLM модель для работы с RAG системой.

## Рекомендуемые модели для локальной работы:

### 1. **Meta Llama 3.1 8B Instruct** (основная рекомендация)
- **Размер**: ~16 GB (в квантизации - 4-8 GB)
- **Производительность**: Отличный баланс качества и скорости
- **Язык**: Хорошая поддержка русского языка
- **Требования**: GPU с 16+ GB VRAM или CPU с 32+ GB RAM

### 2. **Mistral 7B Instruct**
- **Размер**: ~13 GB
- **Производительность**: Быстрее Llama, но чуть хуже качество
- **Язык**: Хорошая мультиязычность
- **Требования**: GPU с 8+ GB VRAM

### 3. **Saiga 7B** (специально для русского)
- **Размер**: ~13 GB
- **Производительность**: Оптимизирована для русского языка
- **Язык**: Отличная поддержка русского
- **Требования**: GPU с 8+ GB VRAM

### 4. **Qwen 2.5 7B Instruct**
- **Размер**: ~14 GB
- **Производительность**: Новая модель с хорошими результатами
- **Язык**: Отличная мультиязычность
- **Требования**: GPU с 8+ GB VRAM

### 5. **Vikhr 7B** (русская модель)
- **Размер**: ~13 GB
- **Производительность**: Создана специально для русского языка
- **Язык**: Только русский
- **Требования**: GPU с 8+ GB VRAM


## Установка необходимых библиотек


In [ ]:
%pip install transformers accelerate bitsandbytes sentencepiece protobuf
%pip install llama-cpp-python  # Для квантизированных моделей (GGUF формат)
%pip install ctransformers      # Альтернативная библиотека для GGUF
%pip install vllm               # Для быстрого инференса (требует GPU)
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118  # CUDA 11.8


## Вариант 1: Скачивание Meta Llama 3.1 8B Instruct (полная версия)


In [5]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Создаем директорию для моделей
os.makedirs("doc_ingest/models/llm", exist_ok=True)

# Название модели на Hugging Face
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# Для доступа к Llama моделям нужен токен Hugging Face
# Получите его на https://huggingface.co/settings/tokens
HF_TOKEN = input("Введите ваш Hugging Face токен (требуется для Llama моделей): ")

print("Скачивание модели Llama 3.1 8B Instruct...")
print("Это может занять 10-30 минут в зависимости от скорости интернета")

# Загрузка токенизатора
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    token=HF_TOKEN,
    cache_dir="doc_ingest/models/llm"
)

# Загрузка модели с квантизацией для экономии памяти
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=HF_TOKEN,
    cache_dir="doc_ingest/models/llm",
    torch_dtype=torch.float16,  # Используем float16 для экономии памяти
    device_map="auto",           # Автоматическое распределение по устройствам
    load_in_8bit=True            # 8-битная квантизация (требует bitsandbytes)
)

# Сохраняем локально
save_path = "doc_ingest/models/llm/llama-3.1-8b-instruct"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"✅ Модель сохранена в: {save_path}")
print(f"Размер на диске: ~{sum(os.path.getsize(os.path.join(save_path, f)) for f in os.listdir(save_path)) / 1e9:.1f} GB")


Скачивание модели Llama 3.1 8B Instruct...
Это может занять 10-30 минут в зависимости от скорости интернета


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct.
403 Client Error. (Request ID: Root=1-68a447d3-181276be5b125b6e2122f884;15f806f0-03ed-4c3e-9be9-ab54924885ac)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/config.json.
Your request to access model meta-llama/Llama-3.1-8B-Instruct is awaiting a review from the repo authors.

## Вариант 2: Скачивание квантизированной версии (GGUF формат) - рекомендуется для CPU


In [2]:
import os
import requests
from tqdm import tqdm

def download_file(url, dest_path):
    """Скачивание файла с прогресс-баром"""
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    
    with open(dest_path, 'wb') as file:
        with tqdm(total=total_size, unit='iB', unit_scale=True) as pbar:
            for data in response.iter_content(chunk_size=1024):
                size = file.write(data)
                pbar.update(size)

# Создаем директорию для квантизированных моделей
os.makedirs("doc_ingest/models/llm/gguf", exist_ok=True)

# Варианты квантизированных моделей (от TheBloke)
models = {
    "llama-3.1-8b-q4": {
        "url": "https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf",
        "size": "4.1 GB",
        "quality": "Хорошее качество, быстрая работа"
    },
    "mistral-7b-q4": {
        "url": "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf", 
        "size": "4.4 GB",
        "quality": "Отличная скорость, хорошее качество"
    },
    "saiga-7b-q4": {
        "url": "https://huggingface.co/IlyaGusev/saiga2_7b_gguf/resolve/main/model-q4_K.gguf",
        "size": "4.1 GB", 
        "quality": "Оптимизирована для русского языка"
    }
}

print("Выберите модель для скачивания:")
for i, (key, info) in enumerate(models.items(), 1):
    print(f"{i}. {key}: {info['size']} - {info['quality']}")

choice = int(input("Введите номер (1-3): ")) - 1
model_key = list(models.keys())[choice]
model_info = models[model_key]

print(f"\nСкачивание {model_key}...")
dest_path = f"doc_ingest/models/llm/gguf/{model_key}.gguf"
download_file(model_info['url'], dest_path)

print(f"✅ Модель сохранена в: {dest_path}")


Выберите модель для скачивания:
1. llama-3.1-8b-q4: 4.1 GB - Хорошее качество, быстрая работа
2. mistral-7b-q4: 4.4 GB - Отличная скорость, хорошее качество
3. saiga-7b-q4: 4.1 GB - Оптимизирована для русского языка

Скачивание saiga-7b-q4...


100%|██████████| 4.08G/4.08G [06:04<00:00, 11.2MiB/s]

✅ Модель сохранена в: doc_ingest/models/llm/gguf/saiga-7b-q4.gguf


## Примеры использования моделей


In [ ]:
# Пример 1: Использование с transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Загрузка модели
model_path = "doc_ingest/models/llm/llama-3.1-8b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Генерация текста
def generate_response(prompt, max_length=200):
    inputs = tokenizer(prompt, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=0.7,
            do_sample=True,
            top_p=0.9
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Тест
prompt = "Извлеки сущности из текста: 'Поставка цемента М500 в количестве 100 тонн на станцию Москва-Товарная до 15 января 2025 года'"
print(generate_response(prompt))


In [13]:
# Пример 2: Использование GGUF модели с llama-cpp-python (для CPU)
from llama_cpp import Llama

# Загрузка квантизированной модели
model_path = "doc_ingest/models/llm/gguf/saiga-7b-q4.gguf"#"doc_ingest/models/llm/gguf/mistral-7b-q4.gguf" 
llm = Llama(
    model_path=model_path,
    n_ctx=2048,  # Размер контекста
    n_threads=8,  # Количество потоков CPU
    n_gpu_layers=0  # Количество слоев на GPU (0 = только CPU)
)
# Генерация
def generate_with_gguf(prompt, max_tokens=2000):
    output = llm(
        prompt,
        max_tokens=max_tokens,
        temperature=0.7,
        top_p=0.9,
        echo=False
    )
    return output['choices'][0]['text']

# Тест извлечения сущностей
prompt = """Извлеки сущности из следующего запроса в формате JSON:
Текст: В начале 2024 года компания “СеверИнвест” объявила о запуске нового проекта по внедрению искусственного интеллекта в управление городской инфраструктурой Санкт-Петербурга. По словам генерального директора компании, Ивана Петрова, основная цель проекта — повысить эффективность использования ресурсов за счёт интеграции IoT-сенсоров, систем предиктивной аналитики и технологий обработки больших данных (Big Data).

Партнёрами выступили международная консалтинговая фирма Accenture и исследовательский центр при Санкт-Петербургском государственном университете. Официальный контракт был подписан 17 марта 2024 года на сумму 2,3 миллиарда рублей. Финансирование обеспечили сразу два источника: Внешэкономбанк и частный фонд Aurora Capital, зарегистрированный в Лондоне.

Проект реализуется в три этапа. На первом этапе (апрель – июнь 2024) устанавливались датчики для мониторинга качества воздуха, транспортных потоков и энергопотребления. В частности, в районе Васильевского острова были смонтированы более 350 устройств, собранных на базе микропроцессоров NVIDIA Jetson Xavier NX. Второй этап (июль – декабрь 2024) предусматривает интеграцию аналитической платформы на основе Apache Kafka, PostgreSQL и фреймворка TensorFlow для предсказания пиковых нагрузок. Третий этап, запланированный на январь – март 2025 года, связан с внедрением систем поддержки принятия решений для городской администрации.

Параллельно с этим проектом в Москве началась реализация программы “Умный транспорт”, курируемой Министерством цифрового развития РФ. В рамках этой инициативы в августе 2024 года был проведён пилот с использованием технологии Computer Vision для распознавания нарушений ПДД. Систему разработала российская компания VisionTech, а тестирование прошло при поддержке ГАИ Москвы. Уже в первые два месяца пилота удалось зафиксировать свыше 12 000 случаев превышения скорости и проезда на красный свет.


Верни JSON с полями: 'intent', 'domain', 'object', 'object_attributes', 'identifiers', 'actors', 'location', 'time', 'quantity', 'price', 'metrics', 'constraints', 'exclusions', 'output_format', 'locale', 'priority', 'authority', 'context_refs', 'sensitivity', 'needs_grounding', 'confidences' """

print(generate_with_gguf(prompt))

llama_model_load_from_file_impl: using device Metal (Apple M1 Pro) - 10916 MiB free
llama_model_loader: loaded meta data with 16 key-value pairs and 291 tensors from doc_ingest/models/llm/gguf/saiga-7b-q4.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32            



Интент: 
- [{"id": 1, "name": "Статья", "description": "Текст"}, {"id": 2, "name": "Результат", "description": "Партнёры"}, {"id": 3, "name": "Источник", "description": "Внешэкономбанк"}, {"id": 4, "name": "Фонд", "description": "Частный фонд Aurora Capital"}, {"id": 5, "name": "Сумма", "description": "2.3 миллиарда рублей"}, {"id": 6, "name": "Место", "description": "Регион Санкт-Петербурга"}, {"id": 7, "name": "Период", "description": "Сентябрь 2024"}, {"id": 8, "name": "Мотивация", "description": "Увеличение эффективности использования ресурсов за счёт интеграции IoT-сенсоров, систем предиктивной аналитики и технологий обработки больших данных (Big Data)"}]

Домен: 
- [{"id": 1, "name": "Структура", "description": "Компания"}, {"id": 2, "name": "Ситуация", "description": "Участие в проекте"}, {"id": 3, "name": "Программа", "description": "Реализация проекта"}, {"id": 4, "name": "Исследовательский центр", "description": "Партнёры"}, {"id": 5, "name": "Создание", "description": "Нач

In [5]:
json_params = {
  "intent": "поиск условий поставки",
  "domain": "Логистика",
  "object": "цемент",
  "object_attributes": ["ГОСТ 31108-2020"],
  "identifiers": ["Договор №12-34/2025"],
  "actors": ["Отдел логистики", "Поставщик X"],
  "location": {"origin":"ст. Ферзиково","destination":"ст. Москва-Тов."},
  "time": {"from": "2025-09-01", "to": None, "deadline": "2025-09-10"},
  "quantity": {"value": 3000, "unit": "тонн"},
  "price": {"amount": None, "currency": "RUB", "vat": "20%"},
  "metrics": ["SLA: отгрузка ≤ 3 дня"],
  "constraints": ["Инкотермс: FCA", "Пломбировка обязательна"],
  "exclusions": ["без авто-доставки"],
  "output_format": "таблица CSV",
  "locale": "ru-RU",
  "priority": "high",
  "authority": ["Политика поставок, п.3.2"],
  "context_refs": [],
  "sensitivity": "normal",
  "needs_grounding": True,
  "confidences": {"domain":0.92,"object":0.88,"incoterms":0.75}
}

In [8]:
json_params.keys()

dict_keys(['intent', 'domain', 'object', 'object_attributes', 'identifiers', 'actors', 'location', 'time', 'quantity', 'price', 'metrics', 'constraints', 'exclusions', 'output_format', 'locale', 'priority', 'authority', 'context_refs', 'sensitivity', 'needs_grounding', 'confidences'])